## ADDITIONAL 
Имеются две базы данных (два листа Excel-файла Data_TSUM.xlsx): база с ценами конкурентов (Data_Parsing) и внутренняя база компании (Data_Company).

В базе парсинга есть два id, однозначно определяющие товар: producer_id и producer_color.

В базе компании есть два аналогичных поля: item_id и color_id.

Нам известно, что коды в двух базах отличаются наличием набора служебных символов. В базе парсинга встречаются следующие символы: _, -, ~, \\\\, /.

Необходимо:

Считать данные из Excel в DataFrame (Data_Parsing) и (Data_Company).
Подтянуть к базе парсинга данные из базы компании (item_id, color_id, current_price) и сформировать столбец разницы цен в % (цена конкурента к нашей цене).
Определить сильные отклонения от среднего в разности цен в пределах бренда-категории (то есть убрать случайные выбросы, сильно искажающие сравнение). Критерий — по вкусу, написать комментарий в коде.
Записать новый файл Excel с базой парсинга, приклееными к ней столбцами из пункта 2 и с учётом пункта 3 (можно добавить столбец outlier и проставить Yes для выбросов).

In [3]:
import pandas as pd
import numpy as np
df_parsing = pd.read_excel('./data/Data_TSUM.xlsx', sheet_name="Data_Parsing')
df_company = pd.read_excel('./data/Data_TSUM.xlsx', sheet_name='Data_Company')


In [4]:
display(df_parsing, df_company)

,brand,Category,producer_id,producer_color,price
0,Valentino,Shoes,aaaaa1111_11,black,167
1,Valentino,Shoes,aaaaa1111_12,black,188
2,Valentino,Shoes,aaaaa1111_13,black,184
3,Valentino,Shoes,aaaaa1111_14,bla//ck,196
4,Valentino,Shoes,aaaaa1111_15,bla\\ck,250
...,...,...,...,...,...
70,Stone Island,Bags,sssss1111_31,~~~red,164
71,Stone Island,Bags,sssss1111_32,~~~red,158
72,Stone Island,Bags,sssss1111_33,~~~red,194
73,Stone Island,Bags,sssss1111_34,~~~red,256


,brand,Category,item_id,color_id,current price
0,Valentino,Shoes,aaaaa111111,black,247
1,Valentino,Shoes,aaaaa111112,black,161
2,Valentino,Shoes,aaaaa111113,black,234
3,Valentino,Shoes,aaaaa111114,black,167
4,Valentino,Shoes,aaaaa111115,black,153
...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,165
71,Stone Island,Bags,sssss111132,red,196
72,Stone Island,Bags,sssss111133,red,236
73,Stone Island,Bags,sssss111134,red,222


In [33]:
# Очистим данные в таблице парсинг от "скверны" в виде спецсимволов

def kill_smb(string):
    return string.replace('_', '').replace('-', '').replace('~', '').replace('\\', '').replace(r'/', '')

df_parsing2 = df_parsing.copy()
df_parsing2['producer_color'] = df_parsing2['producer_color'].apply(kill_smb)
df_parsing2['producer_id'] = df_parsing2['producer_id'].apply(kill_smb)

display(df_parsing2)
    

,brand,Category,producer_id,producer_color,price
0,Valentino,Shoes,aaaaa111111,black,167
1,Valentino,Shoes,aaaaa111112,black,188
2,Valentino,Shoes,aaaaa111113,black,184
3,Valentino,Shoes,aaaaa111114,black,196
4,Valentino,Shoes,aaaaa111115,black,250
...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164
71,Stone Island,Bags,sssss111132,red,158
72,Stone Island,Bags,sssss111133,red,194
73,Stone Island,Bags,sssss111134,red,256


In [42]:
# Мёржим данные из двух датафреймов для анализа
# Выделяем из таблицы компании только нужные поля
df_company_short = df_company[['item_id', 'color_id', 'current price']]
# df_merge = pd.concat([df_parsing, df_company_short], axis=1, join='inner', keys=[])
df_merge = df_parsing2.merge(df_company_short, left_on=['producer_id', 'producer_color'], right_on=['item_id', 'color_id'], how='outer')
display(df_merge)


,brand,Category,producer_id,producer_color,price,item_id,color_id,current price
0,Valentino,Shoes,aaaaa111111,black,167,aaaaa111111,black,247
1,Valentino,Shoes,aaaaa111112,black,188,aaaaa111112,black,161
2,Valentino,Shoes,aaaaa111113,black,184,aaaaa111113,black,234
3,Valentino,Shoes,aaaaa111114,black,196,aaaaa111114,black,167
4,Valentino,Shoes,aaaaa111115,black,250,aaaaa111115,black,153
...,...,...,...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164,sssss111131,red,165
71,Stone Island,Bags,sssss111132,red,158,sssss111132,red,196
72,Stone Island,Bags,sssss111133,red,194,sssss111133,red,236
73,Stone Island,Bags,sssss111134,red,256,sssss111134,red,222


In [59]:
# Добавляем колонки дельта цены, выброс
df_merge['delta'] = df_merge['price']-df_merge['current price']
df_merge['delta_%'] = round(df_merge['price']/df_merge['current price']*100, 2)

display(df_merge)


,brand,Category,producer_id,producer_color,price,item_id,color_id,current price,delta,delta_%
0,Valentino,Shoes,aaaaa111111,black,167,aaaaa111111,black,247,-80,67.61
1,Valentino,Shoes,aaaaa111112,black,188,aaaaa111112,black,161,27,116.77
2,Valentino,Shoes,aaaaa111113,black,184,aaaaa111113,black,234,-50,78.63
3,Valentino,Shoes,aaaaa111114,black,196,aaaaa111114,black,167,29,117.37
4,Valentino,Shoes,aaaaa111115,black,250,aaaaa111115,black,153,97,163.40
...,...,...,...,...,...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164,sssss111131,red,165,-1,99.39
71,Stone Island,Bags,sssss111132,red,158,sssss111132,red,196,-38,80.61
72,Stone Island,Bags,sssss111133,red,194,sssss111133,red,236,-42,82.20
73,Stone Island,Bags,sssss111134,red,256,sssss111134,red,222,34,115.32


In [79]:
display(df_merge.head())
df_merge.drop(['mean_delta'], axis=1, inplace=True)
display(df_merge.head())

,brand,Category,producer_id,producer_color,price,item_id,color_id,current price,delta,delta_%,mean_delta
0,Valentino,Shoes,aaaaa111111,black,167,aaaaa111111,black,247,-80,67.61,99.09
1,Valentino,Shoes,aaaaa111112,black,188,aaaaa111112,black,161,27,116.77,99.09
2,Valentino,Shoes,aaaaa111113,black,184,aaaaa111113,black,234,-50,78.63,99.09
3,Valentino,Shoes,aaaaa111114,black,196,aaaaa111114,black,167,29,117.37,99.09
4,Valentino,Shoes,aaaaa111115,black,250,aaaaa111115,black,153,97,163.40,99.09


,brand,Category,producer_id,producer_color,price,item_id,color_id,current price,delta,delta_%
0,Valentino,Shoes,aaaaa111111,black,167,aaaaa111111,black,247,-80,67.61
1,Valentino,Shoes,aaaaa111112,black,188,aaaaa111112,black,161,27,116.77
2,Valentino,Shoes,aaaaa111113,black,184,aaaaa111113,black,234,-50,78.63
3,Valentino,Shoes,aaaaa111114,black,196,aaaaa111114,black,167,29,117.37
4,Valentino,Shoes,aaaaa111115,black,250,aaaaa111115,black,153,97,163.40


In [78]:
display(df_merge['Category'].unique())
display(df_merge['brand'].unique())

df_merge.groupby(['brand', 'Category'])['delta_%'].mean().unstack()

#display(df_merge[['brand', 'Category'], ['delta_%']].mean())
for b in df_merge['brand'].unique():
    for c in df_merge['Category'].unique():
        m1 = df_merge['brand']==b  
        m2 = df_merge['Category']==c
        mask = m1 & m2
        m=round(df_merge[mask]['delta_%'].mean(),2)
        print(f'Brand = {b}, Category = {c}, mean_delta = {m}')
        #df_merge[mask]['mean_delta'] = m
display(df_merge)



array(['Shoes', 'Bags'], dtype=object)

array(['Valentino', 'Brioni', 'Stone Island'], dtype=object)

Brand = Valentino, Category = Shoes, mean_delta = 100.09
Brand = Valentino, Category = Bags, mean_delta = 102.97
Brand = Brioni, Category = Shoes, mean_delta = 105.3
Brand = Brioni, Category = Bags, mean_delta = 104.05
Brand = Stone Island, Category = Shoes, mean_delta = 104.05
Brand = Stone Island, Category = Bags, mean_delta = 99.09


,brand,Category,producer_id,producer_color,price,item_id,color_id,current price,delta,delta_%,mean_delta
0,Valentino,Shoes,aaaaa111111,black,167,aaaaa111111,black,247,-80,67.61,99.09
1,Valentino,Shoes,aaaaa111112,black,188,aaaaa111112,black,161,27,116.77,99.09
2,Valentino,Shoes,aaaaa111113,black,184,aaaaa111113,black,234,-50,78.63,99.09
3,Valentino,Shoes,aaaaa111114,black,196,aaaaa111114,black,167,29,117.37,99.09
4,Valentino,Shoes,aaaaa111115,black,250,aaaaa111115,black,153,97,163.40,99.09
...,...,...,...,...,...,...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164,sssss111131,red,165,-1,99.39,99.09
71,Stone Island,Bags,sssss111132,red,158,sssss111132,red,196,-38,80.61,99.09
72,Stone Island,Bags,sssss111133,red,194,sssss111133,red,236,-42,82.20,99.09
73,Stone Island,Bags,sssss111134,red,256,sssss111134,red,222,34,115.32,99.09
